In [8]:
from copy import deepcopy
from dataclasses import dataclass
import multiprocessing as mp
from re import T
import torchvision.models as models
import torch
import copy
import torch.nn.functional as F
#from model import net
import random
import multiprocessing as mp
import os
import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from torchvision import datasets,transforms
import random
from torch import optim,ceil
import torch.nn.functional as F
from torch import nn
import pickle
import numpy as np
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [ ]:
from copy import deepcopy
from dataclasses import dataclass
import multiprocessing as mp
from re import T
import torch

In [9]:

from torch.utils.data import Dataset,DataLoader
import numpy as np

class plantdisease(Dataset):
    def __init__(self, df,
                  img_dir='data/oneFolder'
                 , isTest=False):
        self.img_labels=df
        self.img_dir='../../Data/DataResized512/'
       # self.train, self.test = train_test_split(self.img_labels, test_size=0.05)

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
            img_path = os.path.join(self.img_dir, self.img_labels['image'][idx])

            image = (read_image(img_path).float())
            self.transform=transforms.Compose([transforms.Resize((342,512)),
        transforms.Normalize((124.4455, 159.9584, 104.1832), (47.1528, 41.4626, 49.2424))])
            image = self.transform(image)
            label = self.img_labels['labels'][idx]
            label=np.fromstring(label[1:-1], dtype=np.int,sep=',')
            return image, label

In [10]:
def pretty_size(size) -> str:
	"""Pretty prints a torch.Size object"""
	assert(isinstance(size, torch.Size))
	return " × ".join(map(str, size))

def dump_tensors(gpu_only=True):
	"""Prints a list of the Tensors being tracked by the garbage collector."""
	import gc
	total_size = 0
	for obj in gc.get_objects():
		try:
			if torch.is_tensor(obj):
				if not gpu_only or obj.is_cuda:

					total_size += obj.numel()
			elif hasattr(obj, "data") and torch.is_tensor(obj.data):
				if not gpu_only or obj.is_cuda:
					total_size += obj.data.numel()
		except Exception as e:
			pass        
	torch.cuda.empty_cache()
	print("Total size:", total_size)


In [12]:
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()


    dump_tensors()
def save_model(model):
    file_name='testingmodel.h5'
    torch.save(model.state_dict(), file_name)
    return file_name

In [15]:

def confusion_matrixCal(target,pred):
  matrix=0
  for i in range(0,target.shape[0]):
    matrix+=confusion_matrix(target[i], pred[i],labels=[1, 0])
  return matrix

#Test train function

config = {
        "lr": 0.0001,
        "batch_size": 6
        ,"epoch":20
    }
def client_trainGPU(config,run):
  
  # Wandb
  loss_function=torch.nn.BCEWithLogitsLoss()

  wandb.watch(model, loss_function, log="all", log_freq=10)

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  optimizer =optim.Adam(model.parameters(),lr=config["lr"],weight_decay=9e-05)

  train_dataloader=DataLoader(train_data,batch_size=int(config["batch_size"]))
  test_dataloader=DataLoader(test_data,batch_size=int(config["batch_size"]))



  for epoc in range(config['epoch']):
      model.to(device)

      train_loss = 0.0
      confusionMatrix=0
      train_step=0
      print('start Epoch ',epoc)
      for _, datas in enumerate(((train_dataloader)),0):
          data, target=datas
          data,target=data.to(device),target.to(device)
          
          
          output = model(data)

          loss = loss_function(output, target.float())

          train_loss += loss.item()
          train_step+=1


          ouput=torch.sigmoid(output)
          predicted = torch.round(ouput)
          target=target.cpu()
          data=data.cpu()
          predicted=predicted.cpu()
          confusionMatrix+=confusion_matrixCal(target,predicted)
          optimizer.zero_grad(set_to_none=True)
          loss.backward()
          optimizer.step()
      print('Client ID ',  ', epoch ',
        epoc, ': ', train_loss / train_step)

      #Evaluation

      ######
      confusionMatrix_test=0
      test_loss = 0.0
      test_step=0

      best_eval_accuracy=0

      model.cpu()
      model.eval()
      with torch.no_grad():
        for _, datas in enumerate(((test_dataloader)),0):
            data, target=datas
            
            output = model(data)
            
            loss =loss_function(output, target.float())
            ouput=torch.sigmoid(output)
            predicted = np.round(ouput)
            test_loss += loss.item()
            test_step+=1

            confusionMatrix_test+=confusion_matrixCal(target,predicted)
      
      print('test_loss {}'.format(test_loss/test_step))
      train_loss=train_loss / train_step
      test_loss=test_loss/test_step

      TP_test=confusionMatrix_test[0][0]
      FP_test=confusionMatrix_test[0][1]
      FN_test=confusionMatrix_test[1][0]
      TN_test=confusionMatrix_test[1][1]
      accuracy_test=(TP_test+TN_test)/(FP_test+FN_test+TP_test+TN_test)
      recal_test=TP_test/(TP_test+FN_test)
      precision_test=TP_test/(TP_test+FP_test)
      negativePredictiveValue_test=TN_test/(TN_test+FN_test)
      missRate_test=FN_test/(TP_test+FN_test)
      fallOut_test=FP_test/(TN_test+FP_test)
      f1_test=(2*precision_test*recal_test)/(precision_test+recal_test)


      TP_train=confusionMatrix[0][0]
      FP_train=confusionMatrix[0][1]
      FN_train=confusionMatrix[1][0]
      TN_train=confusionMatrix[1][1]
      accuracy_train=(TP_train+TN_train)/(FP_train+FN_train+TP_train+TN_train)
      recal_train=TP_train/(TP_train+FN_train)
      precision_train=TP_train/(TP_train+FP_train)
      negativePredictiveValue_train=TN_train/(TN_train+FN_train)
      missRate_train=FN_train/(TP_train+FN_train)
      fallOut_train=FP_train/(TN_train+FP_train)
      f1_train=(2*precision_train*recal_train)/(precision_train+recal_train)
      if best_eval_accuracy<accuracy_test:
        best_eval_accuracy=accuracy_test
        model_log = wandb.Artifact('best_eval_accuracy', type='model')
        try:
          model_log.add_file(save_model(model))
          run.log_artifact(model_log)
          print('model saved, best accuracy: {}'.format(best_eval_accuracy))
        except:
          print('save model failed')

      wandb.log({
            'epoc': epoc, 
            'train_loss': train_loss,
            'val_loss': test_loss,

            
            'TP_test':TP_test,
            'FP_test':FP_test,
            'FN_test':FN_test,
            'TN_test':TN_test,
            'accuracy_test':accuracy_test,
            'recal_test':recal_test,
            'precision_test':precision_test,
            'negativePredictiveValue_test':negativePredictiveValue_test,
            'f1_test':f1_test,


            'TP':TP_train,
            'FP':FP_train,
            'FN':FN_train,
            'TN':TN_train,
            'accuracy_train':accuracy_train,
            'recal_train':recal_train,
            'precision_train':precision_train,
            'negativePredictiveValue_train':negativePredictiveValue_train,
            'f1_train':f1_train,
          })

  free_gpu_cache()
  print("Finished Training")



In [16]:
os.environ["WANDB_API_KEY"] = '831281456cbf2d04f75207adc2e9811dc4fd32cf'
projName='single_model_training1'
import wandb
def main():

    run = wandb.init()
    
    # note that we define values from `wandb.config` instead 
    # of defining hard values

    config = {
        "lr": 0.00006824,
        "batch_size":16,
        "epoch":20
    }
    import warnings 
    warnings.filterwarnings('ignore')
    a=client_trainGPU(config,run)

    run.finish()
main()


start Epoch  0
Client ID  , epoch  0 :  6.428118379088119
accuracy 0.663625
test_loss 0.11627152686317761
accuracy 0.8174603174603174
start Epoch  1
Client ID  , epoch  1 :  4.212064170918893
accuracy 0.794
test_loss 0.13520628160664014
accuracy 0.7867063492063492
start Epoch  2
Client ID  , epoch  2 :  3.1891820009914227
accuracy 0.84
test_loss 0.13134153672153986
accuracy 0.8125
start Epoch  3
Client ID  , epoch  3 :  2.7146718773874454
accuracy 0.859
test_loss 0.14114743973764163
accuracy 0.8154761904761905
start Epoch  4
Client ID  , epoch  4 :  2.4094277261465322
accuracy 0.874125


# Clear Memory GPU

In [ ]:
def pretty_size(size):
	"""Pretty prints a torch.Size object"""
	assert(isinstance(size, torch.Size))
	return " × ".join(map(str, size))

def dump_tensors(gpu_only=True):
	"""Prints a list of the Tensors being tracked by the garbage collector."""
	import gc
	total_size = 0
	for obj in gc.get_objects():
		try:
			if torch.is_tensor(obj):
				if not gpu_only or obj.is_cuda:
					print("%s:%s%s %s" % (type(obj).__name__, 
										  " GPU" if obj.is_cuda else "",
										  " pinned" if obj.is_pinned else "",
										  pretty_size(obj.size())))
					total_size += obj.numel()
			elif hasattr(obj, "data") and torch.is_tensor(obj.data):
				if not gpu_only or obj.is_cuda:
					print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
												   type(obj.data).__name__, 
												   " GPU" if obj.is_cuda else "",
												   " pinned" if obj.data.is_pinned else "",
												   " grad" if obj.requires_grad else "", 
												   " volatile" if obj.volatile else "",
												   pretty_size(obj.data.size())))
					total_size += obj.data.numel()
		except Exception as e:
			pass        
	print("Total size:", total_size)


Total size: 0


In [ ]:
cf_matrix=0

def confusion_matrix(target,pred):
    for i in range(0,target.shape[0]):
        cf_matrix+=confusion_matrix(target[i], pred[i])
def f1_score(target,pred):
    for i in range(0,target.shape[0]):
       print(accuracy_score(target[i], pred[i]))


NameError: name 'a' is not defined